# K-means text clustering

The grouping of texts into various clusters can be used to differentiate various themes within the texts

# Preparation

In [1]:
#Importing various modules required for analysis
import xlsxwriter 
import scipy.sparse as sp
import numpy as np
import os
os.getcwd()
import tkinter as tk
from tkinter import filedialog
import pandas as pd

Importing the text files for clustering, close the popup window after uploading the file.

In [2]:
#importing the excel. The excel shoud have column named 'Tekst'
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue')
canvas1.pack()
#input training document
def getExcel ():
    global df_train
    
    import_file_path = filedialog.askopenfilename()
    df_train = pd.read_excel (import_file_path)
    #print (df_train)
    
browseButton_Excel = tk.Button(text='Import Excel File', command=getExcel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=browseButton_Excel)

root.mainloop()


check

In [3]:
df_train.head()

,Textnumber,Text
0,1,Determination of the most significant variable...
1,2,Foam Flow Experiments. I. Estimation of the Bu...
2,3,On the transformation behaviour of NiTi partic...
3,4,Brief Communication: The Need for a National-L...
4,5,Preparation of Cu-co-Fe Pre-alloyed powders by...


creating a column named 'Corpus' from the Text

In [4]:
df_train["Corpus"]=df_train["Text"]

## Text tokenizing, lemmatizing, stopword removal

dropping the nan values from text

In [5]:
df_train = df_train[df_train['Corpus'].notna()]

dropping the punctuation from text

In [6]:
import string
string.punctuation
def remove_punctuation(txt):
    txt_nopunct ="".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct
df_train['Corpus_clean']=df_train['Corpus'].apply(lambda x: remove_punctuation(x))

tokenization

In [7]:
import re
def tokenize(txt):
    tokens= re.split('\W+', txt)
    return tokens
df_train['Corpus_clean_tokenized']=df_train['Corpus_clean'].apply(lambda x:tokenize(x.lower()))

removing the stopwords

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
df_train['Corpus_nostop']=df_train['Corpus_clean_tokenized'].apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gebruiker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


lemmatization

In [9]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
wn=nltk.WordNetLemmatizer()
def lemmatization(token_text):
    text= [wn.lemmatize(word) for word in token_text]
    return text
df_train['Corpus_lemmatized']=df_train['Corpus_nostop'].apply(lambda x: lemmatization(x))


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/gebruiker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/gebruiker/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


conversion of columns to lists

In [10]:
train_x=df_train['Corpus_lemmatized'].to_list()


Converting vector values into string values

In [11]:
train_x = [str (item) for item in train_x]

# K-means clustering

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

create vectorizer usingTfidfVectorizer class to fit and transform the document 

In [13]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(train_x)

clusering algorithm

In [14]:
true_k = int(input('type number of clusters you required ',))
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

type number of clusters you required 3


KMeans(max_iter=100, n_clusters=3, n_init=1)

making a list of labels

In [15]:
clusters = model.labels_.tolist()

to get the centroids and features

In [16]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

/home/gebruiker/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


print the centroids into which clusters they belongs

In [17]:
for i in range(true_k):
    print("cluster ", i),
    for ind in order_centroids[i, :20]:
        print(" ", terms[ind])

cluster  0
  water
  experiment
  foam
  flow
  powder
  hydrometallurgical
  process
  cucofe
  preparation
  prealloyed
  estimation
  function
  generationcoalescence
  bubble
  energy
  primary
  metal
  resource
  input
  production
cluster  1
  tool
  diamond
  material
  matrix
  trend
  stone
  processing
  country
  extraction
  ethical
  developing
  real
  raw
  societal
  acceptance
  use
  analysis
  time
  data
  bubble
cluster  2
  higher
  netherlands
  education
  data
  state
  flow
  group
  nationallevel
  need
  research
  working
  communication
  brief
  composite
  estimation
  acceptance
  generationcoalescence
  function
  affecting
  analysis


# Pickling and saving the model

use joblib.dump to pickle the model, once it has converged and to reload the model/reassign the labels as the clusters.

In [18]:
import joblib

uncomment the below code to save your model 

In [19]:
joblib.dump(model,  'doc_cluster.pkl')

['doc_cluster.pkl']

since I've already run my model I am loading from the pickle

In [20]:
model = joblib.load('doc_cluster.pkl')
clusters = model.labels_.tolist()

# using the model to cluster the keywords

In [21]:
df_train['cluster_kmeans'] = clusters

In [22]:
df_train['cluster_kmeans'].value_counts()

0    6
1    3
2    2
Name: cluster_kmeans, dtype: int64

In [23]:
df_train.columns

Index(['Textnumber', 'Text', 'Corpus', 'Corpus_clean',
       'Corpus_clean_tokenized', 'Corpus_nostop', 'Corpus_lemmatized',
       'cluster_kmeans'],
      dtype='object')

In [24]:
#Dropping the columns unncessary for output
df_final=df_train.drop(columns=['Corpus','Corpus_clean', 'Corpus_clean_tokenized',
       'Corpus_nostop', 'Corpus_lemmatized'])

In [25]:
df_final.head()

,Textnumber,Text,cluster_kmeans
0,1,Determination of the most significant variable...,0
1,2,Foam Flow Experiments. I. Estimation of the Bu...,0
2,3,On the transformation behaviour of NiTi partic...,0
3,4,Brief Communication: The Need for a National-L...,2
4,5,Preparation of Cu-co-Fe Pre-alloyed powders by...,0


give name to the output excel

In [26]:
df_final.to_excel('name to the output excel.xlsx')